In [ ]:
!pip install transformers torch av librosa opencv-python-headless --quiet
!pip install transformers torch librosa feat Detector opensmile opencv-python-headless --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2

In [ ]:

import torch
import torch.nn as nn
import numpy as np
from transformers import BertTokenizer, BertModel
import cv2
import opensmile
import librosa
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pandas as pd
import av
from tqdm import tqdm
import subprocess
from scipy.io.wavfile import read as wavread
from io import BytesIO
from IPython.display import Audio
from google.colab import drive
drive.mount('/content/drive')
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
from scipy.stats import pearsonr

Mounted at /content/drive


In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/Dataset_Deeplearning/CMU-MOSI/label.csv',delimiter=';')


In [ ]:
dataset=dataset[dataset['mode']=='train']

In [ ]:
dataset['id_AU']=dataset['video_id'].astype(str)+"_"+dataset['clip_id'].astype(str)

In [ ]:
import pickle

with open("/content/drive/MyDrive/mosi_data.pkl", "rb") as f:
    data = pickle.load(f)

dataset['Auc_video']=list(data['train']['vision'])

ids = []
for i in range(len(data['train']['id'])):
    ids.append(data['train']['id'][i][0])

dataset['Auc_id.pkl']=ids


In [ ]:
import re

In [ ]:
dataset['Auc_id.pkl'] = dataset['Auc_id.pkl'].apply(
    lambda x: x.decode('utf-8') if isinstance(x, bytes) else x
)


In [ ]:
dataset['Auc_id.pkl'] = dataset['Auc_id.pkl'].str.replace(r"b'|'$", "", regex=True)

In [ ]:
dataset['test_final']=dataset['Auc_id.pkl']==dataset['id_AU']

In [ ]:
dataset[dataset['test_final']!=True]

,video_id,clip_id,text,label,label_T,label_A,label_V,annotation,mode,Cle_Audio,Cle_image,id_AU,Auc_video,Auc_id.pkl,test_final


In [ ]:
dataset.shape

(1284, 15)

## La partie Audio

# Model article DEVA

## Données input pour entrainement

In [ ]:
# Fake audio files (sinus wave = simple demo)
sr = 16000

In [ ]:
labels = np.array(list(dataset['label']), dtype=np.float32)



In [ ]:
print("=== Vérification Normalisation ===")
print("mean(labels_norm) =", labels.mean())
print("std(labels_norm)  =", labels.std())
print("min(labels_norm)  =", labels.min())
print("max(labels_norm)  =", labels.max())

=== Vérification Normalisation ===
mean(labels_norm) = 0.23448859
std(labels_norm)  = 1.5094514
min(labels_norm)  = -3.0
max(labels_norm)  = 3.0


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# ----------------------------------------------------------
# 1. MINI DATASET (3 samples)
# ----------------------------------------------------------
annotation=list(dataset['annotation'].str.lower())
texts=list(dataset['text'].str.lower())
labels = torch.tensor(labels).float().to(device)

In [ ]:
images = np.load(
    "/content/drive/MyDrive/Dataset_Deeplearning/train_mosi_frames_list.npy",
    allow_pickle=True
)

In [ ]:
audios=np.load(
    "/content/drive/MyDrive/Dataset_Deeplearning/train_mosi_audio_list.npy",
    allow_pickle=True
)

In [ ]:
total_au=list(dataset['Auc_video'])

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self, T=8, hidden=768):
        super().__init__()

        self.T = T
        self.hidden = hidden

        # special token E_m learnable
        self.Em = nn.Parameter(torch.zeros(1, 1, hidden))

        # small transformer layer
        self.transformer = nn.TransformerEncoderLayer(
            d_model=hidden,
            nhead=8,
            dim_feedforward=2048,
            batch_first=True
        )

        # BERT
        #self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        #self.bert = BertModel.from_pretrained("bert-base-uncased").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
        self.bert = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier").to(device)




    def forward(self, text):

        # tokenize
        tokens = self.tokenizer(text, return_tensors="pt",
                                truncation=True, padding=True).to(device)

        # BERT hidden states (1, L, 768)
        with torch.no_grad():
            bert_seq = self.bert(**tokens).last_hidden_state

        # prepend Em token
        Em = self.Em.expand(1, 1, self.hidden)   # (1,1,768)
        full_seq = torch.cat([Em, bert_seq], dim=1)  # (1, L+1, 768)

        # transformer encoding
        enriched = self.transformer(full_seq)  # (1, L+1, 768)

        # take first T=8 tokens
        Xt = enriched[:, :self.T, :]        # shape = (1, 8, 768)

        # return shape (8, 768)
        return Xt.squeeze(0)

text_encoder = TextEncoder().to(device)



In [ ]:
from transformers import AutoTokenizer, AutoModel

class TextEncoder(nn.Module):
    def __init__(self, T=8, hidden=768):
        super().__init__()

        self.T = T
        self.hidden = hidden

        # special token Em learnable
        self.Em = nn.Parameter(torch.zeros(1, 1, hidden))

        self.transformer = nn.TransformerEncoderLayer(
            d_model=hidden,
            nhead=8,
            dim_feedforward=2048,
            batch_first=True
        )

        model_name = "michellejieli/emotion_text_classifier"

        # Correct tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # ❗ Load AutoModel (not classifier)
        self.bert = AutoModel.from_pretrained(model_name).to(device)

    def forward(self, text):
        tokens = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True
        ).to(device)

        # Hidden states
        with torch.no_grad():
            bert_outputs = self.bert(**tokens)
            bert_seq = bert_outputs.last_hidden_state   # (1, L, 768)

        Em = self.Em.expand(1, 1, self.hidden)
        full_seq = torch.cat([Em, bert_seq], dim=1)

        enriched = self.transformer(full_seq)
        Xt = enriched[:, :self.T, :]

        return Xt.squeeze(0)


text_encoder = TextEncoder().to(device)



Some weights of RobertaModel were not initialized from the model checkpoint at michellejieli/emotion_text_classifier and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ===============================================================
# AUDIO → Englise X_a
# ===============================================================

def encode_audio(audio):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
    mfcc_mean = torch.tensor(mfcc.mean(axis=1)).float().to(device)  # shape: [20]
    return mfcc_mean

# ===============================================================
# AUDIO DESCRIPTION → English sentence D_a
# ===============================================================

def extract_prosodic(audio, sr=16000):
    # Pitch
    f0, _, _ = librosa.pyin(audio, fmin=50, fmax=400, sr=sr)
    pitch = np.nanmean(f0) if np.sum(~np.isnan(f0)) > 0 else 0.0
    # Loudness
    rms = librosa.feature.rms(y=audio)[0]
    loudness = float(np.mean(rms))
    # Jitter (pitch instability)
    valid_f0 = f0[~np.isnan(f0)]
    jitter = float(np.std(np.diff(valid_f0)) / (np.mean(valid_f0) + 1e-6)) if len(valid_f0) > 2 else 0.0
    # Shimmer (amplitude instability)
    shimmer = float(np.std(np.diff(rms)) / (np.mean(rms) + 1e-6)) if len(rms) > 2 else 0.0

    return pitch, loudness, jitter, shimmer




smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals
)

def extract_opensmile(audio, sr=16000):
    result = smile.process_signal(audio, sr)
    return result.values[0]  # 88-dim vector


def audio_description_from_features(pitch, loudness, jitter, shimmer):
    def level(x, low, high):
        if x < low: return "low"
        if x > high: return "high"
        return "moderate"

    pitch_desc   = level(pitch, 120, 220)
    loud_desc    = level(loudness, 0.01, 0.04)
    jitter_desc  = level(jitter, 0.005, 0.015)
    shimmer_desc = level(shimmer, 0.003, 0.01)

    sentence = (
        f"The speaker has a {pitch_desc} pitch, "
        f"a {loud_desc} loudness, "
        f"a {jitter_desc} jitter, and "
        f"a {shimmer_desc} shimmer in their voice."
    )

    return sentence


def compute_AED(audio, sr=16000):
    # -- Prosodic --
    pitch, loud, jitter, shimmer = extract_prosodic(audio, sr)
    # -- openSMILE --
    smile_vec = extract_opensmile(audio, sr)  # 88 dims
    # -- Description text --
    desc = audio_description_from_features(pitch, loud, jitter, shimmer)
    # -- TextEncoder --
    #print(desc)
    D_a = text_encoder(desc).mean(dim=0)  # (768,)

    return D_a

In [ ]:
# ----------------------------------------------------------
# 5. FUSION MODEL
# ----------------------------------------------------------

class FusionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(768 + 20 + 8, 128),
            nn.ReLU(),
            nn.Linear(128,1)
        )
    def forward(self, t, a, v):
        x = torch.cat([t,a,v], dim=-1)
        return self.fc(x)

model = FusionNet().to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()


In [ ]:
# ===============================================================
# Visual DESCRIPTION → visual sentence D_v
# ===============================================================

AU_NAMES = [
    "AU01_inner_brow_raiser",
    "AU02_outer_brow_raiser",
    "AU04_brow_lowerer",
    "AU05_upper_lid_raiser",
    "AU06_cheek_raiser",
    "AU07_lid_tightener",
    "AU09_nose_wrinkler",
    "AU10_upper_lip_raiser",
    "AU12_smile",
    "AU14_dimpler",
    "AU15_lip_corner_depressor",
    "AU17_chin_raiser",
    "AU20_lip_stretcher",
    "AU23_lip_tightener",
    "AU24_lip_presser",
    "AU25_lips_part",
    "AU26_jaw_drop",
    "AU28_lip_suck",
    "AU45_blink",
    "misc"
]

def visual_description(au_vector, threshold=0.3):
    AU = {AU_NAMES[i]: au_vector[i] for i in range(len(AU_NAMES))}
    desc = []

    # Brows
    if AU["AU01_inner_brow_raiser"] > threshold: desc.append("raised inner eyebrows")
    if AU["AU02_outer_brow_raiser"] > threshold: desc.append("raised outer eyebrows")
    if AU["AU04_brow_lowerer"] > threshold: desc.append("lowered eyebrows")

    # Eyes
    if AU["AU05_upper_lid_raiser"] > threshold: desc.append("wide open eyes")
    if AU["AU07_lid_tightener"] > threshold: desc.append("tense eyelids")
    if AU["AU45_blink"] > threshold: desc.append("frequent blinking")

    # Smile / cheeks
    if AU["AU06_cheek_raiser"] > threshold: desc.append("raised cheeks")
    if AU["AU12_smile"] > threshold: desc.append("smiling")
    if AU["AU14_dimpler"] > threshold: desc.append("lip dimpling")

    # Mouth movement
    if AU["AU15_lip_corner_depressor"] > threshold: desc.append("downward lip corners")
    if AU["AU17_chin_raiser"] > threshold: desc.append("raised chin")
    if AU["AU20_lip_stretcher"] > threshold: desc.append("stretched lips")
    if AU["AU23_lip_tightener"] > threshold: desc.append("tightened lips")
    if AU["AU24_lip_presser"] > threshold: desc.append("pressed lips")
    if AU["AU25_lips_part"] > threshold: desc.append("parted lips")
    if AU["AU26_jaw_drop"] > threshold: desc.append("open mouth")
    if AU["AU28_lip_suck"] > threshold: desc.append("lip sucking")

    if len(desc) == 0:
        return "The person shows a neutral facial expression."

    return "The person shows signs of: " + ", ".join(desc) + "."

def keep_nonzero_mean(aus):
    aus = np.array(aus)  # shape = (N_frames, 20)
    valid = aus[~np.all(aus == 0, axis=1)]

    if len(valid) == 0:
        return np.zeros(aus.shape[1])  # vecteur neutre

    return valid.mean(axis=0)

def encode_visual_description(aus):
    # 1. Moyenne des frames non-nulles
    au_mean = keep_nonzero_mean(aus)     # shape (20,)

    # 2. Phrase descriptive
    desc = visual_description(au_mean)
    #print(desc)
    # 3. Encodage avec TON TextEncoder
    D_v = text_encoder(desc).mean(dim=0)

    return D_v


In [ ]:

# ===============================================================
# 4. IMAGE ENCODER (Mini CNN)
# ===============================================================

class ImageEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1, 1))
        )

    def forward(self, video):
        # video : numpy array shape (T, H, W, 3)
        video = torch.tensor(video).float().to(device) / 255.0  # (T, H, W, 3)

        # réorganiser en batch de frames
        video = video.permute(0,3,1,2)  # (T, 3, H, W)

        # CNN sur toutes les frames
        feats = self.cnn(video)         # (T, 64, 1, 1)
        feats = feats.view(video.size(0), -1)  # (T, 64)

        # pooling temporel
        video_feat = feats.mean(dim=0)  # (64,)

        return video_feat

img_encoder = ImageEncoder().to(device)

In [ ]:
class CrossModalAttention(nn.Module):
    def __init__(self, dim_t, dim_m):
        super().__init__()
        self.Wq = nn.Linear(dim_t, dim_m)
        self.Wk = nn.Linear(dim_m, dim_m)
        self.Wv = nn.Linear(dim_m, dim_m)
        self.scale = np.sqrt(dim_m)

    def forward(self, Q, K, V):
        """
        Q : (D_t,)
        K : (D_m,)
        V : (D_m,)
        """
        # ---- Fix 1 : FORCER LE BATCH ----
        if Q.dim() == 1: Q = Q.unsqueeze(0)
        if K.dim() == 1: K = K.unsqueeze(0)
        if V.dim() == 1: V = V.unsqueeze(0)

        # ---- Linear projections ----
        Qp = self.Wq(Q).unsqueeze(1)   # (1,1,Dm)
        Kp = self.Wk(K).unsqueeze(1)   # (1,1,Dm)
        Vp = self.Wv(V).unsqueeze(1)   # (1,1,Dm)

        # ---- Fix 2 : shapes always (1,1,D) ----
        att = torch.softmax((Qp @ Kp.transpose(1, 2)) / self.scale, dim=-1)
        out = att @ Vp                  # (1,1,D)

        return out.squeeze(1).squeeze(0)  # (D,)


In [ ]:
class FeatureEnhancement(nn.Module):
    """
    H = X + FC([X, D])
    """
    def __init__(self, dim_feat, dim_desc, dim_out):
        super().__init__()
        self.fc = nn.Linear(dim_feat + dim_desc, dim_out)

    def forward(self, X, D):
        x = torch.cat([X, D], dim=-1)
        enhanced = self.fc(x)
        return X + enhanced


In [ ]:
class AudioVisualFeatureProjector(nn.Module):
    """
    Projette X_a ou X_v dans un espace aligné (B,1,D)
    """
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.fc = nn.Linear(dim_in, dim_out)

    def forward(self, x):
        x = self.fc(x).unsqueeze(1)
        return x


In [ ]:
class MFU(nn.Module):
    def __init__(self, dim_t=768, dim_mod=256):
        super().__init__()

        # attention T → A et T → V
        self.att_t2a = CrossModalAttention(dim_t, dim_mod)
        self.att_t2v = CrossModalAttention(dim_t, dim_mod)

        # scalaires learnable
        self.alpha = nn.Parameter(torch.tensor(0.5))
        self.beta  = nn.Parameter(torch.tensor(0.5))

    def forward(self, prev_fusion, X_t, X_a, X_v):
        """
        prev_fusion : (B, dim_mod)
        X_t : (B, dim_t)
        X_a : (B, dim_mod)
        X_v : (B, dim_mod)
        """

        att_a = self.att_t2a(X_t, X_a, X_a)
        att_v = self.att_t2v(X_t, X_v, X_v)

        out = prev_fusion + self.alpha * att_a + self.beta * att_v

        return out



In [ ]:
class DEVANet(nn.Module):
    def __init__(self,
                 dim_text=768,
                 dim_audio=20,
                 dim_visual=64,
                 dim_desc=768,
                 dim_hidden=256):

        super().__init__()

        # 1) Audio & Vision projection
        self.audio_proj = AudioVisualFeatureProjector(dim_audio, dim_hidden)
        self.visual_proj = AudioVisualFeatureProjector(dim_visual, dim_hidden)

        # 2) Feature Enhancement
        self.fe_audio  = FeatureEnhancement(dim_hidden, dim_desc, dim_hidden)
        self.fe_visual = FeatureEnhancement(dim_hidden, dim_desc, dim_hidden)

        # ⚠️ TEXTE : XT + D_a + D_v + D_ann  → 3 descriptions donc 3*dim_desc
        self.fe_text   = FeatureEnhancement(dim_text,  dim_desc*3, dim_text)

        # 3) MFU
        self.mfu = MFU(dim_t=dim_text, dim_mod=dim_hidden)

        # 4) Fusion finale
        self.fc = nn.Sequential(
            nn.Linear(dim_hidden, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, X_t, X_a, X_v, D_a, D_v, D_ann):

        # ---- Alignement ----
        A = self.audio_proj(X_a).squeeze(1)   # (256)
        V = self.visual_proj(X_v).squeeze(1)  # (256)

        # ---- Feature Enhancement ----
        H_a0 = self.fe_audio(A, D_a)
        H_v0 = self.fe_visual(V, D_v)

        # ⚠️ TEXTE : concat de 3 descriptions
        D_text = torch.cat([D_a, D_v, D_ann], dim=-1)
        H_t0 = self.fe_text(X_t, D_text)

        # ---- Fusion MFU ----
        fusion = self.mfu(H_a0, H_t0, H_a0, H_v0)

        return self.fc(fusion)



In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
len(text_encoder('Positive').mean(dim=0).to(device))

768

In [ ]:
DEBUG = False   # Active / désactive tous les prints lourds

device = "cuda" if torch.cuda.is_available() else "cpu"

model = DEVANet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

all_preds, all_labels = [], []

for epoch in range(30):
    total_loss = 0
    loop = tqdm(range(len(texts)), desc=f"Epoch {epoch}", ncols=100)

    for i in loop:

        # ---- Encoders externes ----
        X_t = text_encoder(texts[i]).mean(dim=0).to(device)
        X_a = encode_audio(audios[i]).to(device)
        X_v = img_encoder(images[i]).to(device)

        D_a = compute_AED(audios[i]).to(device)
        D_v = encode_visual_description(total_au[i]).to(device)
        D_ann=text_encoder(annotation[i]).mean(dim=0).to(device)

        y = labels[i]
        y = torch.tensor(y, dtype=torch.float32, device=device)

        # ---- Forward ----
        pred = model(X_t, X_a, X_v, D_a, D_v, D_ann).squeeze()
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += abs(loss.item())
        avg_loss = total_loss / len(texts)


        # ---- Affichage optimisé ----
        loop.set_postfix({"batch_loss": f"{loss.item():.3f}"})

        # ---- Stockage pour les métriques ----
        all_preds.append(pred.item())
        all_labels.append(y.item())


    print(f"Epoch {epoch:02d} - MAE = {avg_loss:.3f}")


Epoch 0: 100%|███████████████████████████████| 1284/1284 [10:26<00:00,  2.05it/s, batch_loss=10.334]


Epoch 00 - MAE = 7.183


Epoch 1: 100%|████████████████████████████████| 1284/1284 [10:24<00:00,  2.06it/s, batch_loss=1.588]


Epoch 01 - MAE = 4.223


Epoch 2: 100%|████████████████████████████████| 1284/1284 [10:25<00:00,  2.05it/s, batch_loss=0.800]


...

Epoch 27 - MAE = 1.889


Epoch 28: 100%|███████████████████████████████| 1284/1284 [10:19<00:00,  2.07it/s, batch_loss=0.500]


Epoch 28 - MAE = 1.828


Epoch 29: 100%|███████████████████████████████| 1284/1284 [10:19<00:00,  2.07it/s, batch_loss=0.405]

Epoch 29 - MAE = 1.828


In [ ]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total_params, trainable_params = count_parameters(model)

print(f"📌 Total parameters : {total_params:,}")
print(f"📌 Trainable parameters : {trainable_params:,}")


📌 Total parameters : 3,596,803
📌 Trainable parameters : 3,596,803


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Dataset_Deeplearning/deva_model_30_non_normaliser_annotation.pth")
print("Poids sauvegardés -> deva_model.pth")

Poids sauvegardés -> deva_model.pth


# **Evaluation métrique**

In [ ]:
DF_test=pd.read_csv('/content/drive/MyDrive/Dataset_Deeplearning/CMU-MOSI/label.csv',delimiter=';')


In [ ]:
©=DF_test[DF_test['mode']=='test']

In [ ]:
import pickle

with open("/content/drive/MyDrive/mosi_data.pkl", "rb") as f:
    data = pickle.load(f)

DF_test['Auc_video']=list(data['test']['vision'])

ids = []
for i in range(len(data['test']['id'])):
    ids.append(data['test']['id'][i][0])

DF_test['Auc_id.pkl']=ids

In [ ]:
total_au_eval=list(DF_test['Auc_video'])

In [ ]:
labels = np.array(list(DF_test['label']), dtype=np.float32)




In [136]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# ----------------------------------------------------------
# 1. MINI DATASET (3 samples)
# ----------------------------------------------------------
texts=list(DF_test['text'].str.lower())
annotation=list(DF_test['annotation'].str.lower())
labels = torch.tensor(labels).float().to(device)

In [ ]:
images_test = np.load(
    "/content/drive/MyDrive/Dataset_Deeplearning/test_mosi_frames_list.npy",
    allow_pickle=True
)

In [ ]:
audios_test=np.load(
    "/content/drive/MyDrive/Dataset_Deeplearning/test_mosi_audio_list.npy",
    allow_pickle=True
)

In [137]:
loop = tqdm(range(len(texts)), desc=f"Epoch {epoch}", ncols=100)

labels_prediction=[]
for i in loop:

    # ---- Encoders externes ----
    X_t = text_encoder(texts[i]).mean(dim=0).to(device)
    X_a = encode_audio(audios_test[i]).to(device)
    X_v = img_encoder(images_test[i]).to(device)

    D_a = compute_AED(audios_test[i]).to(device)
    D_v = encode_visual_description(total_au_eval[i]).to(device)
    D_ann=text_encoder(annotation[i]).mean(dim=0).to(device)

    y = labels[i]
    y = torch.tensor(y, dtype=torch.float32, device=device)

    # ---- Forward ----
    pred = model(X_t, X_a, X_v, D_a, D_v,D_ann).squeeze()
    labels_prediction.append(pred.item())


Epoch 29: 100%|███████████████████████████████████████████████████| 686/686 [06:27<00:00,  1.77it/s]


In [138]:
import numpy as np
from sklearn.metrics import f1_score
from scipy.stats import pearsonr

def evaluate_metrics(labels, preds):
    labels = np.array(labels, dtype=float)
    preds  = np.array(preds, dtype=float)

    # -----------------------
    # ACC-2 (binaire)
    # -----------------------
    lab2 = (labels > 0).astype(int)
    pre2 = (preds  > 0).astype(int)
    acc2 = (lab2 == pre2).mean()

    # -----------------------
    # ACC-5 ([-3,3])
    # -----------------------
    def to_5(x):
        if x <= -1.8: return 0
        if x <= -0.6: return 1
        if x <=  0.6: return 2
        if x <=  1.8: return 3
        return 4

    lab5 = np.array([to_5(x) for x in labels])
    pre5 = np.array([to_5(x) for x in preds])
    acc5 = (lab5 == pre5).mean()

    # -----------------------
    # ACC-7 ([-3,3])
    # -----------------------
    def to_7(x):
        if x <= -2.0: return 0
        if x <= -1.0: return 1
        if x <= -0.5: return 2
        if x <=  0.5: return 3
        if x <=  1.0: return 4
        if x <=  2.0: return 5
        return 6

    lab7 = np.array([to_7(x) for x in labels])
    pre7 = np.array([to_7(x) for x in preds])
    acc7 = (lab7 == pre7).mean()

    # -----------------------
    # F1-score (binaire)
    # -----------------------
    f1 = f1_score(lab2, pre2, average='weighted', zero_division=0)

    # -----------------------
    # MAE
    # -----------------------
    mae = np.mean(np.abs(labels - preds))

    # -----------------------
    # Pearson Corr
    # -----------------------
    if len(labels) > 1:
        corr = pearsonr(labels, preds)[0]
    else:
        corr = 0.0

    return acc2, acc5, acc7, f1, mae, corr


In [139]:
acc2, acc5, acc7, f1, mae, corr = evaluate_metrics(all_labels, all_preds)

print("Acc-2:", acc2)
print("Acc-5:", acc5)
print("Acc-7:", acc7)
print("F1:", f1)
print("MAE:", mae)
print("Corr:", corr)

Acc-2: 0.6216510903426792
Acc-5: 0.2694184839044652
Acc-7: 0.19929906542056075
F1: 0.6167868083003565
MAE: 1.2599630779626867
Corr: 0.3066011794704976


# **Inférence**


## **input**




In [140]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [141]:

i=np.random.randint(0,len(texts))
print(i)

342


In [142]:
print("\n====================\n")
print(texts[i])
print("\n===========Ecoutons l'audio associé à la vidéo=========\n")
Audio(audios_test[i], rate=16000)





but the adults were the adults that were actually attempting to watch it and the people who were my age as well were like what did i just see

===========Ecoutons l'audio associé à la vidéo=========



In [143]:
print(labels[0])

tensor(-1.8000, device='cuda:0')


In [147]:
# ===============================================================

i=np.random.randint(0,len(texts))
print("\n====================\n")
print(texts[i])

# ---- Encoders externes ----
X_t = text_encoder(texts[i]).mean(dim=0).to(device)
X_a = encode_audio(audios_test[i]).to(device)
X_v = img_encoder(images_test[i]).to(device)

D_a = compute_AED(audios_test[i]).to(device)
D_v = encode_visual_description(total_au_eval[i]).to(device)
D_ann=text_encoder(annotation[i]).mean(dim=0).to(device)

y = labels[i]
print("Resulatat",y)
y = torch.tensor(y, dtype=torch.float32, device=device)

# ---- Forward ----
pred = model(X_t, X_a, X_v, D_a, D_v,D_ann).squeeze()

print("\n--------------------------------")
print(" Predicted Sentiment Score =", pred)
print(" (range typically -3 = négatif → +3 = positif) ")
print("--------------------------------")




really bad movie
Resulatat tensor(-2.6000, device='cuda:0')

--------------------------------
 Predicted Sentiment Score = tensor(1.5832, device='cuda:0', grad_fn=<SqueezeBackward0>)
 (range typically -3 = négatif → +3 = positif) 
--------------------------------
